In [1]:
import numpy as np
import pandas as pd
import os
import sys
import glob
import time 
import warnings
warnings.filterwarnings("ignore")
import fbprophet
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import current_date
from fbprophet import Prophet
from reduce_mem_usage import reduce_mem_usage
from Xbox_preprocess import Xbox_preprocess
from continuous_data import round_target_column, create_store_test, melt_data

In [3]:
import os
os.chdir('../')

In [3]:
xbox_data = Xbox_preprocess()

In [4]:
data = pd.read_csv('input_files/Xbox_data.csv')
#data = reduce_mem_usage(data)


#convert the sales date into datetime format 

data['Sales Date'] = pd.to_datetime(data['Sales Date'])
data = data.drop_duplicates()

closed_stores = ['Saturn Connect Trier', 'Media Markt Heilbronn 2','Saturn Schweinfurt Schrammstraße', 'Saturn Connect Köln',
                    'Saturn Stuttgart-Hbf', 'Saturn-Berlin Clayallee','Saturn Mönchengladbach - Stresemannstraße', 'Saturn Lübeck',
                    'Saturn München Theresienhöhe', 'Saturn Berlin-Alt-Treptow','Media Markt Turnstraße', 'Saturn Wiesbaden Bahnhofsplatz',
                    'Media Markt Ellwangen - DAS', 'Media Markt GmbH Nürtingen','Media Markt Meppen', 'Media Markt Schleswig', 
                    'Media-Saturn IT Services GmbH', 'Meida Markt Waiblingen', 'Saturn Bergisch Gladbach','Saturn Wesel', 'Saturn Hagen', 
                    'Media Markt Bad Cannstatt', 'Saturn Heidelberg', 'Saturn Hildesheim', 'Saturn Münster am York-Ring', 'Media Markt Köln-Chorweiler',
                    'Saturn Dessau', 'Saturn Essen-Steele','Saturn Euskirchen', 'Saturn Göttingen', 'Saturn Hennef', 'Saturn Herford',
                    'Saturn Düsseldorf','Saturn Itzehoe','Saturn Siegburg','Saturn Weiterstadt', 'Saturn Bremerhaven - BSS', 'Saturn Gelsenkirchen Buer']
data = data.loc[~data['Store_names'].isin(closed_stores)].reset_index(drop=True)


#data = (data.set_index("Sales Date").groupby(['Reseller City','Super Division', 'Business Unit', pd.Grouper(freq='W')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())
data['black_week'] = np.where((data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.day > 23), 1, 0)

max_date = data['Sales Date'].max()

In [5]:
max_date

Timestamp('2022-02-26 00:00:00')

In [6]:
data.head()

,Sales Date,Rslr Sales Quantity,Store_names,Business Unit,Rslr Sales Amount,Super Division,Product Division,Reseller Postal Code,Reseller City,black_week
0,2017-01-01,0.0,Saturn Köln-Weiden,Game Pass PC,0.0,EDG Managed - Xbox Live,Xbox LIVE Gold,50858,Köln,0
1,2017-01-01,0.0,Saturn Köln-Weiden,Xbox Series S,0.0,EDG Managed - Xbox Hardware,Xbox Console,50858,Köln,0
2,2017-01-01,0.0,Saturn Köln-Weiden,Xbox Series X,0.0,EDG Managed - Xbox Hardware,Xbox Console,50858,Köln,0
3,2017-01-01,0.0,Fil. Media Markt Würzburg City,Game Pass PC,0.0,EDG Managed - Xbox Live,Xbox LIVE Gold,97070,Würzburg,0
4,2017-01-01,0.0,Fil. Media Markt Würzburg City,Xbox Series S,0.0,EDG Managed - Xbox Hardware,Xbox Console,97070,Würzburg,0


In [7]:
data.nunique()

Sales Date              1759
Rslr Sales Quantity      331
Store_names              398
Business Unit              3
Rslr Sales Amount       8360
Super Division             2
Product Division           2
Reseller Postal Code     379
Reseller City            261
black_week                 2
dtype: int64

In [8]:
data.shape

(2100246, 10)

In [9]:
import calendar as cal
from datetime import *
from dateutil.relativedelta import *
import holidays

def black_week(year): 
    black_friday = (datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4)) - timedelta(days=4)), datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4))+ timedelta(days=1)))
    cyber_monday = datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4))+ timedelta(days=3))

    black_friday = pd.to_datetime(black_friday)
    black_friday =pd.date_range(black_friday[0], black_friday[1], freq ='d')

    cyber_monday = pd.to_datetime(cyber_monday)
    #cyber_monday = list(cyber_monday)


    return black_friday, cyber_monday

In [10]:
# ## create the test_dataframe 

test_data = create_store_test(data, max_date)

test_data['Sales Date'] = pd.to_datetime(test_data['Sales Date'])
test_data['black_week'] = np.where((test_data['Sales Date'].dt.month==11) & (test_data['Sales Date'].dt.day > 23), 1, 0)



In [11]:
test_data.nunique()

Super Division            2
Product Division          2
Business Unit             3
Store_names             398
Rslr Sales Amount         1
Sales Date               33
Rslr Sales Quantity       1
Reseller City           261
Reseller Postal Code    379
black_week                1
dtype: int64

In [12]:
test_data.shape

(39402, 10)

In [13]:
data = pd.concat([data, test_data])

data['Sales Date'] = pd.to_datetime(data['Sales Date'])

In [14]:
data['Sales Date'].max()

Timestamp('2022-03-31 00:00:00')

In [15]:
data = (data.set_index("Sales Date").groupby(['Store_names','Reseller City','Super Division', 'Product Division','Business Unit', pd.Grouper(freq='M')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())

In [16]:
data['black_week'] = np.where(data['Sales Date'].dt.month==11, 1, 0)

In [17]:
data

,Store_names,Reseller City,Super Division,Product Division,Business Unit,Sales Date,Rslr Sales Quantity,Rslr Sales Amount,black_week
0,Saturn Köln-Weiden,Köln,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,2017-01-31,16,4215,0
1,Saturn Köln-Weiden,Köln,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,2017-02-28,0,0,0
2,Saturn Köln-Weiden,Köln,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,2017-03-31,10,2645,0
3,Saturn Köln-Weiden,Köln,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,2017-04-30,14,3636,0
4,Saturn Köln-Weiden,Köln,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,2017-05-31,1,247,0
...,...,...,...,...,...,...,...,...,...
72829,Saturn Zwickau,Zwickau,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,2021-09-30,4,105,0
72830,Saturn Zwickau,Zwickau,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,2021-10-31,9,227,0
72831,Saturn Zwickau,Zwickau,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,2022-01-31,0,0,0
72832,Saturn Zwickau,Zwickau,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,2022-02-28,2,55,0


In [18]:
import plotly.graph_objs as go
def plot_fig(forecast):
    
    forecast = (forecast.set_index("Sales Date").groupby(['Super Division', 'Product Division','Business Unit', pd.Grouper(freq='M')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())
    forecast1 = forecast.loc[forecast['Business Unit']=='Xbox Series S']
    forecast2 = forecast.loc[forecast['Business Unit']=='Xbox Series X']
    forecast3 = forecast.loc[forecast['Business Unit']=='Game Pass PC']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast1['Sales Date'], y=forecast1['Rslr Sales Quantity'], name='Xbox Series S',))
    fig.add_trace(go.Scatter(x=forecast2['Sales Date'], y=forecast2['Rslr Sales Quantity'], name='Xbox Series X',))
    fig.add_trace(go.Scatter(x=forecast3['Sales Date'], y=forecast3['Rslr Sales Quantity'], name='Game Pass PC',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['promos'], name='promos',))
    fig.show()

In [19]:
store_data = data.loc[data['Store_names'] == 'Media Markt Freiburg'].reset_index(drop=True)

In [20]:
def black_week(data):
    data['black_week'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2017 |2018 |2019), 2, data['black_week'])
    data['black_week'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2020), 2, data['black_week'])
    data['black_week'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2021), 0, data['black_week'])
    data['black_week'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==11), 0, data['black_week'])

    data['christmas'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2017), 2, 0)
    data['christmas'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2018 |2019 |2021), 1, data['christmas'])
    data['christmas'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2020 |2021), 1, data['christmas'])
    data['christmas'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2017), 2, data['christmas'])
    data['christmas'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2018 |2019), 2, data['christmas'])
    return data


In [21]:
data['black_week'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2017 |2018 |2019), 2, data['black_week'])
data['black_week'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2020), 2, data['black_week'])
data['black_week'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2021), 0, data['black_week'])
data['black_week'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==11), 0, data['black_week'])

In [22]:
data['christmas'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2017), 2, 0)
data['christmas'] = np.where((data['Business Unit']=='Xbox Series S') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2018 |2019 |2021), 1, data['christmas'])
data['christmas'] = np.where((data['Business Unit']=='Xbox Series X') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2020 |2021), 1, data['christmas'])
data['christmas'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2017), 2, data['christmas'])
data['christmas'] = np.where((data['Business Unit']=='Game Pass PC') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2018 |2019), 2, data['christmas'])

In [23]:
store_data = black_week(store_data)

In [24]:
import time 
spark = SparkSession.builder\
        .appName('xbox_forecasting') \
        .master('local[*]') \
        .config('spark.sql.execution.arrow.pyspark.enabled', True) \
        .config('spark.sql.execution.arrow.enabled', True) \
        .getOrCreate()
         #.config('spark.sql.repl.eagerEval.enabled', True) \
         #.config('spark.ui.showConsoleProgress', True) \
         #.config('spark.default.parallelism', 1308) \



start_time = time.time()

sdf = spark.createDataFrame(data)
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))
sdf = sdf.withColumnRenamed('Sales Date', 'ds')\
        .withColumnRenamed('Super Division', 'Super_Division')\
        .withColumnRenamed('Product Division', 'Product_Division')\
        .withColumnRenamed('Rslr Sales Quantity', 'Rslr_Sales_Qunatity')\
        .withColumnRenamed('Reseller City', 'Reseller_City')\
        .withColumnRenamed('Business Unit', 'Business_Unit')

sdf.printSchema()

sdf.createOrReplaceTempView('sales')

0.05 min: Lags
root
 |-- Store_names: string (nullable = true)
 |-- Reseller_City: string (nullable = true)
 |-- Super_Division: string (nullable = true)
 |-- Product_Division: string (nullable = true)
 |-- Business_Unit: string (nullable = true)
 |-- ds: timestamp (nullable = true)
 |-- Rslr_Sales_Qunatity: integer (nullable = true)
 |-- Rslr Sales Amount: integer (nullable = true)
 |-- black_week: integer (nullable = true)
 |-- christmas: integer (nullable = true)



In [25]:
spark.sql("select Store_names, Reseller_City,  Business_Unit, count(*) from sales group by Store_names, Reseller_City, Business_Unit order by Reseller_City, Business_Unit").show()

sql = 'SELECT Store_names, Reseller_City, Super_Division, Product_Division, Business_Unit, black_week, christmas, ds, sum(Rslr_Sales_Qunatity) as y FROM sales GROUP BY Store_names, Reseller_City, Super_Division, Product_Division, Business_Unit, black_week, christmas, ds ORDER BY Store_names, Reseller_City,  Super_Division,  Business_Unit, ds'

sdf.explain()

+--------------------+-------------+-------------+--------+
|         Store_names|Reseller_City|Business_Unit|count(1)|
+--------------------+-------------+-------------+--------+
|  Media Markt Aachen|       Aachen| Game Pass PC|      61|
|       Saturn Aachen|       Aachen| Game Pass PC|      61|
|  Media Markt Aachen|       Aachen|Xbox Series S|      61|
|       Saturn Aachen|       Aachen|Xbox Series S|      61|
|  Media Markt Aachen|       Aachen|Xbox Series X|      61|
|       Saturn Aachen|       Aachen|Xbox Series X|      61|
|   Media Markt Aalen|        Aalen| Game Pass PC|      61|
|   Media Markt Aalen|        Aalen|Xbox Series S|      61|
|   Media Markt Aalen|        Aalen|Xbox Series X|      61|
|Media Markt Berli...|  Ahrensfelde| Game Pass PC|      61|
|Media Markt Berli...|  Ahrensfelde|Xbox Series S|      61|
|Media Markt Berli...|  Ahrensfelde|Xbox Series X|      61|
|Media Markt Albstadt|     Albstadt| Game Pass PC|      61|
|Media Markt Albstadt|     Albstadt|Xbox

In [26]:
### make partitions of data based on number of cores in the local system
sdf.rdd.getNumPartitions()

spark.sql(sql).show()

store_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['Store_names','Business_Unit'])).cache()
store_part.explain()

+-------------------+-------------+--------------------+----------------+-------------+----------+---------+-------------------+---+
|        Store_names|Reseller_City|      Super_Division|Product_Division|Business_Unit|black_week|christmas|                 ds|  y|
+-------------------+-------------+--------------------+----------------+-------------+----------+---------+-------------------+---+
| Saturn Köln-Weiden|         Köln|EDG Managed - Xbo...|    Xbox Console|Xbox Series S|         0|        0|2017-01-31 00:00:00| 16|
| Saturn Köln-Weiden|         Köln|EDG Managed - Xbo...|    Xbox Console|Xbox Series S|         0|        0|2017-02-28 00:00:00|  0|
| Saturn Köln-Weiden|         Köln|EDG Managed - Xbo...|    Xbox Console|Xbox Series S|         0|        0|2017-03-31 00:00:00| 10|
| Saturn Köln-Weiden|         Köln|EDG Managed - Xbo...|    Xbox Console|Xbox Series S|         0|        0|2017-04-30 00:00:00| 14|
| Saturn Köln-Weiden|         Köln|EDG Managed - Xbo...|    Xbox Cons

In [27]:
### make the resultant schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('Store_names',StringType()),
    StructField('Reseller_City',StringType()),
    StructField('Super_Division',StringType()),
    StructField('Product_Division',StringType()),
    StructField('Business_Unit',StringType()),
    StructField('y',DoubleType()),
    StructField('yhat',DoubleType()),
    StructField('yhat_upper',DoubleType()),
    StructField('yhat_lower',DoubleType())
])

In [28]:
### create the holiday dataframe 

lockdown1 = pd.date_range('2020-03-22', '2020-05-03', freq ='m').to_list()
lockdown2 = pd.date_range('2020-12-13', '2021-03-07', freq ='m').to_list()
lockdown = lockdown1+lockdown2


lock_down = pd.DataFrame({
    'holiday': 'lock_down',
    'ds' : pd.to_datetime(lockdown)})


In [29]:
##### city-wise prophet function 
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import itertools

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_sales( store_pd):

    black_week = dict(zip(store_pd['ds'], store_pd['black_week']))
    christmas = dict(zip(store_pd['ds'], store_pd['christmas']))

    train = store_pd[store_pd['ds']<='2022-02-28']
    future_pd = store_pd[store_pd['ds']>'2022-02-28'].set_index('ds')

    train['date_index'] = train['ds']
    train['date_index'] = pd.to_datetime(train['date_index'])
    train = train.set_index('date_index')


    # param_grid = {  
    #         'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    #         'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    #     }

    # # Generate all combinations of parameters
    # all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    # rmses = []  # Store the RMSEs for each params here
    # cutoffs = pd.date_range(start='2021-01-01', end='2021-12-31', freq='2MS')

    # for params in all_params:
    #     model = Prophet(interval_width=0.95, holidays = lock_down, **params)
    #     model.add_country_holidays(country_name='DE')
    #     model.add_regressor('black_week')
    #     model.add_regressor('christmas')
    #     model.fit(train[['ds', 'y', 'black_week', 'christmas']])

    #     df_cv = cross_validation(model, cutoffs=cutoffs, horizon='60 days', parallel="processes")
    #     df_p = performance_metrics(df_cv, rolling_window=1)
    #     rmses.append(df_p['rmse'].values[0])

    # best_params = all_params[np.argmin(rmses)]

    best_model = Prophet(interval_width=0.95, holidays = lock_down)
                         
    best_model.add_country_holidays(country_name='DE')
    best_model.add_regressor('black_week')
    best_model.add_regressor('christmas')
    best_model.fit(train[['ds', 'y', 'black_week', 'christmas']])

    future = best_model.make_future_dataframe(periods=1, freq='m')
    future['black_week'] = future['ds'].map(black_week)
    future['christmas'] = future['ds'].map(christmas)
 
    #future['black_week'] = np.where(future['ds'].dt.month==11, 1, 0)

    forecast_pd = best_model.predict(future[['ds', 'black_week', 'christmas']])  


    #forecast_pd = model.predict(future_pd[['ds', 'black_week', 'EOL_Promos']])  

    f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')

    #store_pd = store_pd.filter(store_pd['ds']<'2021-10-01 00:00:00')
    st_pd = store_pd[[ 'ds', 'Store_names', 'Reseller_City', 'Super_Division', 'Product_Division', 'Business_Unit','y']].set_index('ds')
    #st_pd = pd.concat([st_pd1, st_pd2])

    results_pd = f_pd.join( st_pd, how='left' )
    results_pd.reset_index(level=0, inplace=True)

    #results_pd[['Reseller_City','Business_Unit']] = future_pd[['Reseller_City','Business_Unit']].iloc[0]

    return results_pd[ ['ds', 'Store_names', 'Reseller_City', 'Super_Division', 'Product_Division', 'Business_Unit','y', 'yhat', 'yhat_upper', 'yhat_lower'] ]

In [30]:
results = (
    store_part
    .groupBy(['Store_names','Business_Unit'])
    .apply(forecast_sales)
    .withColumn('training_date', current_date() )
    )

### cache the results 
start_time = time.time()
results.cache()
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

results.explain()
results = results.coalesce(1)

0.00 min: Lags
== Physical Plan ==
InMemoryTableScan [ds#368, Store_names#369, Reseller_City#370, Super_Division#371, Product_Division#372, Business_Unit#373, y#374, yhat#375, yhat_upper#376, yhat_lower#377, training_date#388]
   +- InMemoryRelation [ds#368, Store_names#369, Reseller_City#370, Super_Division#371, Product_Division#372, Business_Unit#373, y#374, yhat#375, yhat_upper#376, yhat_lower#377, training_date#388], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) Project [ds#368, Store_names#369, Reseller_City#370, Super_Division#371, Product_Division#372, Business_Unit#373, y#374, yhat#375, yhat_upper#376, yhat_lower#377, 2022-03-09 AS training_date#388]
            +- FlatMapGroupsInPandas [Store_names#0, Business_Unit#75], forecast_sales(Store_names#0, Reseller_City#64, Super_Division#31, Product_Division#42, Business_Unit#75, black_week#8, christmas#9, ds#20, y#162L), [ds#368, Store_names#369, Reseller_City#370, Super_Division#371, Product_Division#372, B

In [31]:
### convert the result from sparkdataframe to panadas datafrme 
start_time = time.time()
final_df = results.toPandas()

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

56.16 min: Lags


In [32]:
final_df['yhat'] = np.where(final_df['yhat']<0, 0, final_df['yhat'])
final_df['yhat_upper'] = np.where(final_df['yhat_upper']<0, 0, final_df['yhat_upper'])
final_df['yhat_lower'] = np.where(final_df['yhat_lower']<0, 0, final_df['yhat_lower'])

In [33]:
final_df = final_df.drop_duplicates()
final_df.to_csv(r'output_files/Xbox_store_historic.csv', index=False)

predicted_data = final_df.loc[final_df['ds']> '2022-02-28']

predicted_data = round_target_column(predicted_data, 'yhat')
predicted_data = round_target_column(predicted_data, 'yhat_lower')
predicted_data = round_target_column(predicted_data, 'yhat_upper')

predicted_data.to_csv(r'output_files/Xbox_store_monthly.csv', index=False)

In [28]:
test_results = final_df.loc[(final_df['ds']>'2021-12-31') & (final_df['ds']<'2022-02-28')]

In [29]:
test_results.head()

,ds,Store_names,Reseller_City,Super_Division,Product_Division,Business_Unit,y,yhat,yhat_upper,yhat_lower,training_date
60,2022-01-31,MM Stade,Stade,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,0.0,3.911857,12.728252,0.0,2022-02-11
122,2022-01-31,MM Stade,Stade,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series S,14.0,6.432873,16.757293,0.0,2022-02-11
184,2022-01-31,Media Markt Aachen,Aachen,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series X,1.0,0.000000,5.217307,0.0,2022-02-11
246,2022-01-31,Media Markt Albstadt,Albstadt,EDG Managed - Xbox Live,Xbox LIVE Gold,Game Pass PC,0.0,0.000000,6.381105,0.0,2022-02-11
308,2022-01-31,Media Markt Alzey,Alzey,EDG Managed - Xbox Hardware,Xbox Console,Xbox Series X,0.0,0.000000,6.284357,0.0,2022-02-11


In [30]:
from sklearn.metrics import mean_absolute_error
rmse_pred = mean_absolute_error(test_results['y'], test_results['yhat'])
    
print("Root Mean Absolute Error_store:" , np.sqrt(rmse_pred))

Root Mean Absolute Error_store: 2.512841004511788


In [31]:
normalize_rmse = np.sqrt(rmse_pred)/(test_results['y'].max()-test_results['y'].min())
print("Normalize RMSE:" , normalize_rmse)

Normalize RMSE: 0.0017572314716865652


In [42]:
stores = final_df['Store_names'].unique()
product=final_df['Business_Unit'].unique()

In [43]:
import plotly.graph_objs as go
def plot_fig(df, store, item):
    fig = go.Figure()
    # Create and style traces
    forecast = df.loc[df['Store_names'] == store]
    print(store, item)
    forecast = forecast.loc[forecast['Business_Unit'] == item]
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['y'], name='Actual',))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], name='lowelimit',))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], name='upperlimit',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['promos'], name='promos',))
    fig.show()

In [60]:
plot_fig(final_df, stores[396], product[1])

Saturn Flensburg Xbox Series S
